In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import time

# Đọc DataFrame 'transaction'
transaction_df = pd.read_csv('../balanced_transaction-from-to-prefix-22-KLTN-newtime.csv')
print(transaction_df.shape)

(47463, 48)


In [2]:
# loading the lstm model
lstm_model = tf.keras.models.load_model('./Result-ML/final/lstm')
sgan_model = tf.keras.models.load_model('./Result-ML/final/disc_sup_0010.h5')

# Tiền xử lý dữ liệu
replace_nan = transaction_df.replace(np.nan, -1)

X = replace_nan.drop(columns=['hash','from_address','to_address','label'], axis=1).values

##### reshape SGAN #####
# Padding giá trị -2 để có số lượng đặc trưng là bội số của 49
num_padding = 49 - X.shape[1] % 49
X_padded = np.pad(X, ((0, 0), (0, num_padding)), mode='constant', constant_values=-2)

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(X_padded)

X_reshaped_sgan = normalized_data.reshape(-1, 7, 7)

##### reshape LSTM #####
# Padding giá trị -2 để có số lượng đặc trưng là bội số của 49
num_padding = 900 - X.shape[1] % 900
X_padded = np.pad(X, ((0, 0), (0, num_padding)), mode='constant', constant_values=-2)

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(X_padded)

X_reshaped_lstm = normalized_data.reshape(-1, 30, 30)

#dự đoán
y_pred_sgan=sgan_model.predict(X_reshaped_sgan)
y_pred_sgan=np.argmax(y_pred_sgan,axis=1)
y_pred_lstm=lstm_model.predict(X_reshaped_lstm)
y_pred_lstm=np.argmax(y_pred_lstm,axis=1)


# Kiểm tra xem số lượng dự đoán có khớp với số hàng trong transaction_df
if len(y_pred_lstm) == len(transaction_df) and len(y_pred_sgan) == len(transaction_df):
    # Thêm cột mới vào transaction_df
    transaction_df['lstm_prediction'] = y_pred_lstm
    transaction_df['sgan_prediction'] = y_pred_sgan
else:
    print("Số lượng dự đoán không khớp với số hàng trong DataFrame. Hãy kiểm tra lại.")

# Hiển thị một vài hàng đầu tiên để kiểm tra
print(transaction_df[['hash', 'label', 'lstm_prediction', 'sgan_prediction']].head())
print(transaction_df.shape)

1484/1484 [==============================] - 16s 10ms/step
                                                hash  label  lstm_prediction   
0  0x1d8dc0228d0ecc674c00dbc8c02dfc31e235ee1f2577...      1                1  \
1  0xdf3a1a30c2914b306dd8cb45b5227232daa29f809bfe...      0                0   
2  0xc4120db5c02a21116ad9e9dabc9dafa6ef2ef6e6e4e3...      0                0   
3  0xbe7c8d401d352a3b26364c02e2771d8869cd3c2e4269...      0                0   
4  0xa0a022845546742aa76ceb49b6b77b55df373c9f6467...      1                1   

   sgan_prediction  
0                1  
1                0  
2                0  
3                0  
4                1  
(47463, 50)


In [3]:
######### Convert manual #########
# Chuyển đổi DataFrame thành mảng NumPy

replace_nan = transaction_df.replace(np.nan, -1)

X = replace_nan.drop(columns=['hash','from_address','to_address','label'], axis=1).values

# Padding giá trị -2 để có số lượng đặc trưng là bội số của 49
num_padding = 49 - X.shape[1] % 49
X_padded = np.pad(X, ((0, 0), (0, num_padding)), mode='constant', constant_values=-2)

# Tiền xử lý dữ liệu
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(X_padded)

# Reshape mảng thành ma trận 3 chiều 4x4
X_reshaped = normalized_data.reshape(-1, 7, 7)

print(X_reshaped[0])

######### Convert image #########
# image_arrays = data_df['image'].tolist()
# # Tạo numpy array từ danh sách các mảng 2D
# np_images = np.stack(image_arrays, axis=0)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, transaction_df['label'].values, test_size=0.2, random_state=42)
# print(type(X_train))
# print(type(y_train))

[[9.81037152e-02 2.89604821e-01 9.90888919e-02 1.64745564e-01
  1.98023835e-02 3.52191235e-01 1.03305785e-01]
 [5.09593767e-06 1.23760689e-03 5.17312733e-04 7.93691456e-23
  6.21459592e-04 9.58204334e-02 4.86963579e-03]
 [8.27284011e-04 1.96981912e-02 0.00000000e+00 4.91450481e-04
  4.80016893e-04 2.89604736e-01 3.77547433e-07]
 [0.00000000e+00 1.11242759e-01 2.89604824e-01 1.02923700e-01
  3.70677520e-01 4.82081161e-03 3.87484641e-02]
 [2.49713870e-03 5.04349160e-06 2.53652007e-05 2.92663004e-05
  0.00000000e+00 7.31399700e-04 2.50379056e-01]
 [6.41445215e-03 2.38282480e-03 4.80286476e-02 3.03030303e-21
  9.88033052e-04 3.65578858e-04 2.89604736e-01]
 [3.16621317e-07 0.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]]


In [4]:
# Required Libraries
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, LeakyReLU, Dropout, Flatten, Lambda, Activation, Reshape, Conv2DTranspose, Conv1DTranspose, Conv1D
# from keras.utils.vis_utils import plot_model
from keras.datasets.mnist import load_data
from keras import backend as K
from tensorflow.keras.optimizers import Adam
from numpy.random import randn, randint
from numpy import expand_dims, zeros, ones, asarray
from matplotlib import pyplot



# load the images
def load_real_samples(n_classes=10):
    # (trainX, trainy), (_, _) = load_data()
    trainX, _, trainy, _ = train_test_split(X_reshaped, transaction_df['label'].values, test_size=0.2, random_state=42)

    # print(trainX.shape, trainy.shape)
    # print(type(trainX))
    # print(trainX[0])
    # print(type(trainX[0][0]))
    # print(type(trainy))
    # print(trainy[2])
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    trainy = trainy.astype('uint8')
    # scale from [0,255] to [0,1] as we will be using tanh activation.
#     X = X / 255.0

    # Convert pandas to numpy for trainy
    # trainy = trainy.values
    # print(X[0])
    # print(type(X))
    # print(X.shape, trainy.shape)
    return [X, trainy]

data = load_real_samples()



In [ ]:
from numpy import expand_dims, zeros, ones, asarray
from numpy.random import randn, randint

from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model, Sequential

from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.layers import LeakyReLU, Dropout, Lambda, Activation
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

#from keras.utils import to_categorical

from matplotlib import pyplot as plt
from keras import backend as K
import numpy as np

np.random.seed(42)

# define the standalone generator model
def define_generator(latent_dim):

	in_lat = Input(shape=(latent_dim,))
	#Start with enough dense nodes to be reshaped and ConvTransposed to 28x28x1
	n_nodes = 256 * 7 * 7
	X = Dense(n_nodes)(in_lat)
	X = LeakyReLU(alpha=0.2)(X)
	X = Reshape((7, 7, 256))(X)

	X = Conv2DTranspose(128, (2,2), strides=(1,1), padding='same')(X) #14x14x128
	X = LeakyReLU(alpha=0.2)(X)

	X = Conv2DTranspose(64, (2,2), strides=(1,1), padding='same')(X) #14x14x64
	X = LeakyReLU(alpha=0.2)(X)
	# output
	out_layer = Conv2DTranspose(1, (2,2), strides=(1,1), activation='sigmoid',
                             padding='same')(X) #28x28x1
	# define model
	model = Model(in_lat, out_layer)
	return model

# gen_model=define_generator(100)
# print(gen_model.summary())

# define the base discriminator model for sup and unsup discriminators
#This is the base discriminator that supervised and unsupervised are going to share weights from.
#(I know that the above ssentence is bad english!!!)
def define_discriminator(in_shape=(7,7,1), n_classes=2):
    in_image = Input(shape=in_shape)
    X = Conv2D(32, (2,2), strides=(2,2), padding='same')(in_image)
    X = LeakyReLU(alpha=0.2)(X)

    X = Conv2D(64, (2,2), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)

    X = Conv2D(128, (2,2), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)

    X = Flatten()(X)
    X = Dropout(0.4)(X) #Consider adding more dropout layers to minimize overfitting - remember we work with limited labeled data.
    X = Dense(n_classes)(X)

    model = Model(inputs=in_image, outputs=X)

    return model

#Define the supervised discriminator.
#Multiclass classification, so we will use softmax activation.
#To avoid converting our labels to categorical, we will work with sparse categorical crossentropy loss.
def define_sup_discriminator(disc):
    model=Sequential()
    model.add(disc)
    model.add(Activation('sigmoid'))
    #Let us use sparse categorical loss so we dont have to convert our Y to categorical
    model.compile(optimizer=Adam(learning_rate=0.0001, beta_1=0.5),
                  loss="sparse_categorical_crossentropy",metrics=['accuracy'])
    return model


#Define the unsupervised discriminator
#Takes the output of the supervised, just before the softmax activation.
#Then, adds a layer with calculation of sum of exponential outputs. (defined below as custom_activation)
# Reference: https://arxiv.org/abs/1606.03498

#This custom activation layer gives a value close to 0 for smaller activations
#in the prior discriminator layer. It gives values close to 1 for large activations.
#This way it gives low activation for fake images. No need for sigmoid anymore.

# custom activation function for the unsupervised discriminator
#D(x) = Z(x) / (Z(x) + 1) where Z(x) = sum(exp(l(x))). l(x) is the output from sup discr. prior to softmax
def custom_activation(x):
    Z_x = K.sum(K.exp(x), axis=-1, keepdims=True)
    D_x = Z_x /(Z_x+1)

    return D_x

#You can also try the built in tensorflow function: tf.reduce_logsumexp(class_logits, 1)

def define_unsup_discriminator(disc):
    model=Sequential()
    model.add(disc)
    model.add(Lambda(custom_activation))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001, beta_1=0.5))
    return model


# disc=define_discriminator()
# disc_sup=define_sup_discriminator(disc)
# disc_unsup=define_unsup_discriminator(disc)
# print(disc_unsup.summary())


# define the combined generator and discriminator model, for updating the generator
def define_gan(gen_model, disc_unsup):

	disc_unsup.trainable = False # make unsup. discriminator not trainable
	gan_output = disc_unsup(gen_model.output) #Gen. output is the input to disc.
	model = Model(gen_model.input, gan_output)
	model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001, beta_1=0.5))
	return model

# gan_model = define_gan(gen_model, disc_unsup)
# print(gan_model.summary())

#select subset of the dataset for supervised training
#Let us pick only 100 samples to be used in supervised training.
#Also, we need to ensure we pick 10 samples per class to ensure a good balance
#of data between classes.
def select_supervised_samples(dataset, n_samples=100, n_classes=2):
 	print('test select')
 	X, y = dataset
 	X_list, y_list = list(), list()
 	n_per_class = int(n_samples / n_classes) #Number of amples per class.
 	for i in range(n_classes):
         X_with_class = X[y == i] # get all images for this class
         ix = np.random.randint(0, len(X_with_class), n_per_class) # choose random images for each class
         print(ix)
         [X_list.append(X_with_class[j]) for j in ix] # add to list
         [y_list.append(i) for j in ix]
 	return asarray(X_list), asarray(y_list) #Returns a list of 2 numpy arrays corresponding to X and Y


# Pick real samples from the dataset.
#Return both images and corresponding labels in addition to y=1 indicating that the images are real.
#Remember that we will not use the labels for unsupervised, only used for supervised.
def generate_real_samples(dataset, n_samples):

	images, labels = dataset
	ix = randint(0, images.shape[0], n_samples)
	X, labels = images[ix], labels[ix] #Select random images and corresponding labels
	y = ones((n_samples, 1)) #Label all images as 1 as these are real images. (for the discriminator training)
	return [X, labels], y

# generate latent points, to be used as inputs to the generator.
def generate_latent_points(latent_dim, n_samples):
	z_input = randn(latent_dim * n_samples)
	z_input = z_input.reshape(n_samples, latent_dim) # reshape for input to the network
	return z_input

# Generate fake images using the generator and above latent points as input to it.
#We do not care about labeles so the generator will not know anything about the labels.
def generate_fake_samples(generator, latent_dim, n_samples):

	z_input = generate_latent_points(latent_dim, n_samples)
	fake_images = generator.predict(z_input)
	# create class labels
	y = zeros((n_samples, 1)) #Label all images as 0 as these are fake images. (for the discriminator training)
	return fake_images, y

# report accuracy and save plots & the model periodically.
def summarize_performance(step, bat_per_epo, gen_model, disc_sup, latent_dim, dataset, n_samples=100):
	global max_accuracy
	# Generate fake images
	X, _ = generate_fake_samples(gen_model, latent_dim, n_samples)

# 	X = (X + 1) / 2.0 # scale to [0,1] for plotting
# 	# plot images
# 	for i in range(100):
# 		plt.subplot(10, 10, 1 + i)
# 		plt.axis('off')
# 		plt.imshow(X[i, :, :, 0], cmap='gray_r')
# 	# save plot to drive
# 	filename1 = 'generated_plot_%04d.png' % (step+1)
# 	plt.savefig(filename1)
# 	plt.close()

	# evaluate the discriminator
	X, y = dataset
	_, acc = disc_sup.evaluate(X, y, verbose=0)
    
    # Lấy thời gian bắt đầu
	start_time = time.perf_counter()
    
	y_pred=disc_sup.predict(X_test)
    
	# Lấy thời gian kết thúc
	end_time = time.perf_counter()
	# Tính thời gian thực thi (đơn vị: giây)
	execution_time = end_time - start_time
	# Chuyển đơi sang đơn vị micro giây
	execution_time_microseconds = execution_time * 1000000
	print(f"Thời gian thực thi: {execution_time_microseconds:.2f} micro giây")

	y_pred=np.argmax(y_pred,axis=1)
	accuracy = accuracy_score(y_test, y_pred)
	if accuracy > max_accuracy:
		max_accuracy = acc
        
		f1 = f1_score(y_test, y_pred)
		tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
		# Tính toán dương tính giả (false positive rate)
		fpr = fp / (fp + tn)

		print(f"Dương tính giả (false positive rate): {fpr:.4f}")
		with open('max_accuracy-SGAN-LSTM.txt', 'w') as file:
			# Ghi nội dung vào file
			file.write('Epoch ' + str(((step+1) / bat_per_epo)) + ' - Accuracy: ' + str(max_accuracy) 
            + '- F1_score: ' + str(f1) + '- FPR: ' + str(fpr))
	print('Discriminator Accuracy: %.3f%%' % (acc * 100))
	# save the generator model
# 	filename2 = './Result-ML/SGAN-255/Test_20/gen_model_%04d.h5' % (step+1)
# 	gen_model.save(filename2)
	# save the Discriminator (classifier) model
# 	filename3 = './Result-ML/sgan-lstm/disc_sup_%04d.h5' % ((step+1) / bat_per_epo)
# 	disc_sup.save(filename3)
# 	print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

# train the generator and discriminator
def train(gen_model, disc_unsup, disc_sup, gan_model, dataset, latent_dim, n_epochs=20, n_batch=32):

    # select supervised dataset for training.
    #Remember that we are not using all 60k images, just a subset (100 images, 10 per class. )
	X_sup, y_sup = select_supervised_samples(dataset)
	#print(X_sup.shape, y_sup.shape)

	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	# iterations
	n_steps = bat_per_epo * n_epochs

	half_batch = int(n_batch / 2)
	print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs,
                                                              n_batch, half_batch,
                                                              bat_per_epo, n_steps))

    #  enumerate epochs
	for i in range(n_steps):
		# update supervised discriminator (disc_sup) on real samples.
        #Remember that we use real labels to train as this is supervised.
        #This is the discriminator we really care about at the end.
        #Also, this is a multiclass classifier, not binary. Therefore, our y values
        #will be the real class labels for MNIST. (NOT 1 or 0 indicating real or fake.)
		[Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
		sup_loss, sup_acc = disc_sup.train_on_batch(Xsup_real, ysup_real)

		# update unsupervised discriminator (disc_unsup) - just like in our regular GAN.
        #Remember that we will not train on labels as this is unsupervised, just binary as in our regular GAN.
        #The y_real below indicates 1s telling the discriminator that these images are real.
        #do not confuse this with class labels.
        #We will discard this discriminator at the end.
		[X_real, _], y_real = generate_real_samples(dataset, half_batch) #
		d_loss_real = disc_unsup.train_on_batch(X_real, y_real)
        #Now train on fake.
		X_fake, y_fake = generate_fake_samples(gen_model, latent_dim, half_batch)
		d_loss_fake = disc_unsup.train_on_batch(X_fake, y_fake)

		# update generator (gen) - like we do in regular GAN.
        #We can discard this model at the end as our primary goal is to train a multiclass classifier (sup. disc.)
		X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
		gan_loss = gan_model.train_on_batch(X_gan, y_gan)

		# summarize loss on this batch
		print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, sup_loss, sup_acc*100, d_loss_real, d_loss_fake, gan_loss))
		# evaluate the model performance periodically
		if (i+1) % (bat_per_epo * 1) == 0:
			summarize_performance(i, bat_per_epo, gen_model, disc_sup, latent_dim, dataset)

#################################################################################
# TRAIN
#################################

latent_dim = 100

# create the discriminator models
disc=define_discriminator() #Bare discriminator model...
disc_sup=define_sup_discriminator(disc) #Supervised discriminator model
disc_unsup=define_unsup_discriminator(disc) #Unsupervised discriminator model.

gen_model = define_generator(latent_dim) #Generator
gan_model = define_gan(gen_model, disc_unsup) #GAN
dataset = load_real_samples() #Define the dataset by loading real samples. (This will be a list of 2 numpy arrays, X and y)

max_accuracy = 0

# train the model
# NOTE: 1 epoch = 600 steps in this example.
train(gen_model, disc_unsup, disc_sup, gan_model, dataset, latent_dim, n_epochs=8000, n_batch=32)



test select
[15795   860  5390 11964 11284  6265 16850  4426 14423 11363 16023  8322
  1685   769  2433  5311  5051  6420 17568 19769  6396  8666 18942 18431
  2747   189 19118  3005  1899  1267 17912 11394  3556  3890  8838 14502
 10627  8792 10555 10253  8433 10233 11016  2612 15787 17159 12206  8226
 14541  3152]
[ 1585  3943  1021 11653 10805 13417  7989  9692 12990  6873  5675   161
  4297   995 11534  7629  1016  8529 17262  9268 12185  6331  8571  7208
  5276 16448 16216  8006  2568  2027  2695 15422  5258  6736   391 13986
 12666  5892  3561  6184  8392 13067 15265  2454 11837 14039 10965  9762
  5056 14948]
n_epochs=8000, n_batch=32, 1/2=16, b/e=1186, steps=9488000
1/1 [==============================] - 0s 75ms/step
>1, c[0.698,38], d[0.411,1.078], g[0.416]
1/1 [==============================] - 0s 18ms/step
>2, c[0.694,44], d[0.408,1.070], g[0.422]
1/1 [==============================] - 0s 28ms/step
>3, c[0.691,75], d[0.407,1.071], g[0.423]
1/1 [==============================

1/1 [==============================] - 0s 13ms/step
>80, c[0.423,100], d[0.408,0.928], g[0.505]
1/1 [==============================] - 0s 20ms/step
>81, c[0.448,100], d[0.372,0.916], g[0.505]
1/1 [==============================] - 0s 11ms/step
>82, c[0.460,94], d[0.421,0.921], g[0.514]
1/1 [==============================] - 0s 18ms/step
>83, c[0.426,94], d[0.395,0.915], g[0.509]
1/1 [==============================] - 0s 22ms/step
>84, c[0.443,94], d[0.406,0.916], g[0.515]
1/1 [==============================] - 0s 4ms/step
>85, c[0.412,100], d[0.396,0.902], g[0.525]
1/1 [==============================] - 0s 18ms/step
>86, c[0.358,100], d[0.392,0.914], g[0.523]
1/1 [==============================] - 0s 20ms/step
>87, c[0.410,100], d[0.386,0.904], g[0.525]
1/1 [==============================] - 0s 17ms/step
>88, c[0.387,100], d[0.398,0.898], g[0.530]
1/1 [==============================] - 0s 11ms/step
>89, c[0.375,94], d[0.395,0.891], g[0.530]
1/1 [==============================] - 0s 19m

1/1 [==============================] - 0s 20ms/step
>165, c[0.222,94], d[0.479,0.466], g[0.964]
1/1 [==============================] - 0s 17ms/step
>166, c[0.058,100], d[0.233,0.451], g[1.005]
1/1 [==============================] - 0s 16ms/step
>167, c[0.038,100], d[0.153,0.478], g[0.999]
1/1 [==============================] - 0s 18ms/step
>168, c[0.407,88], d[0.285,0.493], g[1.048]
1/1 [==============================] - 0s 15ms/step
>169, c[0.049,100], d[0.237,0.477], g[1.050]
1/1 [==============================] - 0s 16ms/step
>170, c[0.058,100], d[0.296,0.470], g[1.070]
1/1 [==============================] - 0s 17ms/step
>171, c[0.026,100], d[0.225,0.428], g[1.060]
1/1 [==============================] - 0s 18ms/step
>172, c[0.069,100], d[0.255,0.406], g[1.052]
1/1 [==============================] - 0s 4ms/step
>173, c[0.099,94], d[0.260,0.431], g[1.109]
1/1 [==============================] - 0s 17ms/step
>174, c[0.044,100], d[0.174,0.404], g[1.089]
1/1 [=============================

1/1 [==============================] - 0s 22ms/step
>250, c[0.006,100], d[0.300,0.215], g[1.778]
1/1 [==============================] - 0s 19ms/step
>251, c[0.114,94], d[0.124,0.182], g[1.891]
1/1 [==============================] - 0s 18ms/step
>252, c[0.011,100], d[0.207,0.212], g[1.833]
1/1 [==============================] - 0s 16ms/step
>253, c[0.003,100], d[0.137,0.163], g[1.813]
1/1 [==============================] - 0s 17ms/step
>254, c[0.004,100], d[0.213,0.205], g[1.926]
1/1 [==============================] - 0s 13ms/step
>255, c[0.003,100], d[0.100,0.192], g[1.893]
1/1 [==============================] - 0s 16ms/step
>256, c[0.011,100], d[0.120,0.161], g[1.896]
1/1 [==============================] - 0s 20ms/step
>257, c[0.011,100], d[0.213,0.167], g[1.876]
1/1 [==============================] - 0s 18ms/step
>258, c[0.009,100], d[0.278,0.186], g[1.889]
1/1 [==============================] - 0s 18ms/step
>259, c[0.018,100], d[0.111,0.187], g[1.859]
1/1 [==========================

>334, c[0.002,100], d[0.068,0.113], g[2.446]
1/1 [==============================] - 0s 17ms/step
>335, c[0.002,100], d[0.125,0.088], g[2.459]
1/1 [==============================] - 0s 3ms/step
>336, c[0.017,100], d[0.060,0.123], g[2.418]
1/1 [==============================] - 0s 19ms/step
>337, c[0.002,100], d[0.078,0.084], g[2.359]
1/1 [==============================] - 0s 33ms/step
>338, c[0.003,100], d[0.068,0.086], g[2.473]
1/1 [==============================] - 0s 11ms/step
>339, c[0.108,94], d[0.151,0.098], g[2.490]
1/1 [==============================] - 0s 3ms/step
>340, c[0.002,100], d[0.076,0.094], g[2.495]
1/1 [==============================] - 0s 17ms/step
>341, c[0.034,100], d[0.080,0.092], g[2.519]
1/1 [==============================] - 0s 16ms/step
>342, c[0.001,100], d[0.080,0.091], g[2.569]
1/1 [==============================] - 0s 17ms/step
>343, c[0.022,100], d[0.218,0.086], g[2.477]
1/1 [==============================] - 0s 4ms/step
>344, c[0.001,100], d[0.110,0.109]

...

1/1 [==============================] - 0s 20ms/step
>6877, c[0.012,100], d[0.673,0.693], g[0.791]
1/1 [==============================] - 0s 20ms/step
>6878, c[0.007,100], d[0.705,0.645], g[0.900]
1/1 [==============================] - 0s 18ms/step
>6879, c[0.005,100], d[0.640,0.619], g[0.839]
1/1 [==============================] - 0s 23ms/step
>6880, c[0.180,94], d[0.670,0.568], g[0.857]
1/1 [==============================] - 0s 16ms/step
>6881, c[0.005,100], d[0.681,0.648], g[0.747]
1/1 [==============================] - 0s 20ms/step
>6882, c[0.012,100], d[0.758,0.621], g[0.747]
1/1 [==============================] - 0s 18ms/step
>6883, c[0.005,100], d[0.650,0.626], g[0.835]
1/1 [==============================] - 0s 18ms/step
>6884, c[0.207,94], d[0.627,0.690], g[0.819]
1/1 [==============================] - 0s 33ms/step
>6885, c[0.012,100], d[0.653,0.677], g[0.780]
1/1 [==============================] - 0s 15ms/step
>6886, c[0.006,100], d[0.797,0.591], g[0.764]
1/1 [=================

1/1 [==============================] - 0s 19ms/step
>6961, c[0.174,94], d[0.638,0.704], g[0.704]
1/1 [==============================] - 0s 17ms/step
>6962, c[0.011,100], d[0.748,0.850], g[0.783]
1/1 [==============================] - 0s 19ms/step
>6963, c[0.014,100], d[0.673,0.626], g[0.791]
1/1 [==============================] - 0s 18ms/step
>6964, c[0.011,100], d[0.782,0.747], g[0.738]
1/1 [==============================] - 0s 22ms/step
>6965, c[0.142,94], d[0.592,0.712], g[0.832]
1/1 [==============================] - 0s 17ms/step
>6966, c[0.011,100], d[0.703,0.596], g[0.797]
1/1 [==============================] - 0s 16ms/step
>6967, c[0.191,94], d[0.670,0.833], g[0.814]
1/1 [==============================] - 0s 32ms/step
>6968, c[0.018,100], d[0.607,0.718], g[0.748]
1/1 [==============================] - 0s 22ms/step
>6969, c[0.004,100], d[0.593,0.674], g[0.777]
1/1 [==============================] - 0s 19ms/step
>6970, c[0.016,100], d[0.698,0.732], g[0.736]
1/1 [==================

1/1 [==============================] - 0s 5ms/step
>7045, c[0.199,94], d[0.650,0.687], g[0.747]
1/1 [==============================] - 0s 22ms/step
>7046, c[0.004,100], d[0.726,0.594], g[0.796]
1/1 [==============================] - 0s 16ms/step
>7047, c[0.211,94], d[0.589,0.771], g[0.737]
1/1 [==============================] - 0s 15ms/step
>7048, c[0.164,94], d[0.603,0.770], g[0.810]
1/1 [==============================] - 0s 16ms/step
>7049, c[0.019,100], d[0.645,0.713], g[0.824]
1/1 [==============================] - 0s 19ms/step
>7050, c[0.018,100], d[0.763,0.723], g[0.728]
1/1 [==============================] - 0s 19ms/step
>7051, c[0.007,100], d[0.618,0.643], g[0.833]
1/1 [==============================] - 0s 20ms/step
>7052, c[0.011,100], d[0.693,0.638], g[0.742]
1/1 [==============================] - 0s 19ms/step
>7053, c[0.008,100], d[0.681,0.642], g[0.781]
1/1 [==============================] - 0s 15ms/step
>7054, c[0.008,100], d[0.710,0.678], g[0.791]
1/1 [===================

1/1 [==============================] - 0s 16ms/step
>7127, c[0.011,100], d[0.680,0.656], g[0.761]
1/1 [==============================] - 0s 15ms/step
>7128, c[0.199,94], d[0.689,0.720], g[0.702]
1/1 [==============================] - 0s 18ms/step
>7129, c[0.007,100], d[0.690,0.617], g[0.760]
1/1 [==============================] - 0s 16ms/step
>7130, c[0.012,100], d[0.696,0.672], g[0.799]
1/1 [==============================] - 0s 16ms/step
>7131, c[0.003,100], d[0.586,0.607], g[0.836]
1/1 [==============================] - 0s 16ms/step
>7132, c[0.008,100], d[0.640,0.637], g[0.754]
1/1 [==============================] - 0s 16ms/step
>7133, c[0.009,100], d[0.695,0.716], g[0.792]
1/1 [==============================] - 0s 16ms/step
>7134, c[0.009,100], d[0.716,0.628], g[0.794]
1/1 [==============================] - 0s 17ms/step
>7135, c[0.003,100], d[0.711,0.648], g[0.795]
1/1 [==============================] - 0s 16ms/step
>7136, c[0.011,100], d[0.636,0.718], g[0.784]
1/1 [================

1/1 [==============================] - 0s 24ms/step
>7211, c[0.021,100], d[0.675,0.702], g[0.848]
1/1 [==============================] - 0s 23ms/step
>7212, c[0.008,100], d[0.596,0.681], g[0.769]
1/1 [==============================] - 0s 16ms/step
>7213, c[0.018,100], d[0.607,0.722], g[0.782]
1/1 [==============================] - 0s 16ms/step
>7214, c[0.011,100], d[0.635,0.683], g[0.855]
1/1 [==============================] - 0s 16ms/step
>7215, c[0.011,100], d[0.666,0.687], g[0.810]
1/1 [==============================] - 0s 18ms/step
>7216, c[0.009,100], d[0.845,0.629], g[0.807]
1/1 [==============================] - 0s 16ms/step
>7217, c[0.016,100], d[0.624,0.669], g[0.822]
1/1 [==============================] - 0s 16ms/step
>7218, c[0.010,100], d[0.698,0.683], g[0.780]
1/1 [==============================] - 0s 16ms/step
>7219, c[0.015,100], d[0.598,0.752], g[0.715]
1/1 [==============================] - 0s 16ms/step
>7220, c[0.000,100], d[0.557,0.634], g[0.798]
1/1 [===============